In [ ]:
# Librerie utili per la manipolazione dati
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Librerie utili per i modelli
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report
import seaborn as sns
# Importazione modello utilizzato
from sklearn.ensemble import RandomForestClassifier
# Per la memorizzazione e il riuso di modelli allenati
import joblib
# Importazione grid search
from sklearn.model_selection import GridSearchCV

In [ ]:
# Path file di output dove andare a scrivere i risultati

# TrackNet
# result = "C:/Users/hp/Desktop/TennisAnalyticsGit/Models/result/TrackNet/model_result_*_int*_RF.txt"

# MoveNet
result = "C:/Users/hp/Desktop/TennisAnalyticsGit/Models/result/MoveNet/model_result_n*_*_RF.txt"

In [ ]:
# Apertura file in scrittura
f = open(result, 'w+')

In [ ]:
# Lettura del dataset

# TrackNet
# dataset = pd.read_csv("C:/Users/hp/Desktop/TennisAnalyticsGit/Models/dataset/TrackNet/dataset_*_int*.csv")

# MoveNet
dataset = pd.read_csv("C:/Users/hp/Desktop/TennisAnalyticsGit/Models/dataset/MoveNet/dataset_n*_*.csv")

In [ ]:
# Anteprima dataset
dataset.head()

In [ ]:
# Numero di righe
len(dataset)

In [ ]:
# Numero di colonne
len(dataset.columns)

In [ ]:
# Numero di colonne - 1 perchè si parte da 0 (non si conta la colonna finale 'Shot')
index_last_column = len(dataset.columns) - 1

In [ ]:
index_last_column

In [ ]:
# Vengono prese tutte le righe e tutte le colonne a eccezione dell'ultima quindi abbiamo un vettore n x m-1
# X sono i campioni!
X = dataset.iloc[:, :index_last_column]

In [ ]:
X.shape

In [ ]:
# Vengono prese tutte le righe e solo l'ultima colonna quindi abbiamo un vettore n x 1
# y sono i target!
y = dataset.iloc[:, index_last_column]

In [ ]:
y.shape

In [ ]:
# Per avere un'idea di quanti diritti e rovesci sono stati etichettati 
diritti = 0
rovesci = 0 
for tmp in y:
    if tmp == 1:
        diritti += 1
    if tmp == 2:
        rovesci += 1

In [ ]:
diritti

In [ ]:
rovesci

In [ ]:
# Si suddivide il dataset in modo tale da avere un 70% di TR e un 30% di TS
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# PER AVERE UN'IDEA DELLE METRICHE DA UTILIZZARE PER VALUTARE IL MODELLO
# sklearn.metrics.SCORERS.keys()
# dict_keys(['explained_variance', 'r2', 'max_error', 'matthews_corrcoef', 'neg_median_absolute_error', 'neg_mean_absolute_error',
#             'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error',
#               'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 
#               'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss',
#                 'neg_brier_score', 'positive_likelihood_ratio', 'neg_negative_likelihood_ratio', 'adjusted_rand_score', 'rand_score', 
#                 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 
#                 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 
#                 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted',
#                   'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples',
#                     'jaccard_weighted'])

In [ ]:
# Inizializzazione modello
RF = RandomForestClassifier()

In [ ]:
# GRID SEARCH

In [ ]:
# Make a dictionary of hyperparameter values to search
search_space_RF = {
    "n_estimators" : [10, 50, 100, 200],
    "max_features" : [10, 50, 100, 200],
    "max_depth" : [5, 10, 20, 30]
}
# 64 modelli

In [ ]:
# Make a GridSerachCV object
GS = GridSearchCV(estimator = RF,
                  param_grid = search_space_RF,
                  scoring = ["accuracy", "precision", "recall", "f1"], # sklearn.metrics.SCORES.keys()
                  refit = "accuracy", # quando ci sono classi bilanciate
                  cv = 3,
                  verbose = 4
)

In [ ]:
# Allenamento modello con una K-fold CV
GS.fit(X_train, y_train)

In [ ]:
# INFO UTILI SUL MIGLIOR MODELLO RITORNATO

In [ ]:
# To get the colpete details of the best models
print(GS.best_estimator_)

In [ ]:
# To get only the best hyperparameter values that we searched for
print(GS.best_params_)

In [ ]:
# To get the best score according to the metric we passed in refit
print(GS.best_score_)

In [ ]:
# Si prende il miglior modello restituito
RF = GS.best_estimator_

In [ ]:
# Per salvare il modello allenato
# joblib.dump(RF,'./storemodels/TrackNet/RF_4_5.joblib')
joblib.dump(RF,'./storemodels/MoveNet/RF_n1_5.joblib')

In [ ]:
# Predizioni effettuate modello sui dati di test
y_pred = RF.predict(X_test)

In [ ]:
y_pred

In [ ]:
# INFORMAZIONI SUI DATI DI TRAINING

In [ ]:
# Per ottenere l'accuracy sui dati di training
accuracy_train_GS = GS.score(X_train, y_train)

In [ ]:
# Oppure
accuracy_train_RF = RF.score(X_train, y_train)

In [ ]:
accuracy_train_GS

In [ ]:
accuracy_train_RF

In [ ]:
# Modo alternativo per ottenere l'accuracy sui dati di training

In [ ]:
# Predizioni effettuate modello sui dati di training
y_train_pred = RF.predict(X_train)

In [ ]:
accuracy_train = accuracy_score(y_train, y_train_pred)

In [ ]:
accuracy_train

In [ ]:
# FINE INFORMAZIONI DATI DI TRAINING

In [ ]:
# Valutazione prestazioni del modello
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=0)

accuracy_train = accuracy_score(y_train, y_train_pred)
report_train = classification_report(y_train, y_train_pred, zero_division=0)

In [ ]:
# Stampa dei risultati
print(f'Accuracy: {round(accuracy_train,5)}')
print('\nClassification Report Train:')
print(report_train)
print('\nConfusion Matrix Train:')
print(metrics.confusion_matrix(y_train, y_train_pred))

# Stampa dei risultati
print(f'Accuracy: {round(accuracy,5)}')
print('\nClassification Report Test:')
print(report)
print('\nConfusion Matrix Test:')
print(metrics.confusion_matrix(y_test, y_pred))

confusion_matrix_train = metrics.confusion_matrix(y_train, y_train_pred)
confusion_matrix_train_df = pd.DataFrame(confusion_matrix_train, index=range(2)) # per shot che vale 1/2

confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
confusion_matrix_df = pd.DataFrame(confusion_matrix, index=range(2)) # per shot che vale 1/2

# fmt='g' serve per evitare la stampa in notazione esponenziale come ad esemio 2.5e+02 invece di 250
axes = sns.heatmap(confusion_matrix_train_df, annot=True, cmap='viridis', fmt='g', xticklabels=["1", "2"], yticklabels=["1", "2"]) # per shot che vale 1/2
plt.title('Confusion Matrix')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.show()

# fmt='g' serve per evitare la stampa in notazione esponenziale come ad esemio 2.5e+02 invece di 250
axes = sns.heatmap(confusion_matrix_df, annot=True, cmap='viridis', fmt='g', xticklabels=["1", "2"], yticklabels=["1", "2"]) # per shot che vale 1/2
plt.title('Confusion Matrix')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.show()

In [ ]:
# Scrittura dei risultati sul file
f.write('---------------------------- RF --------------------------')
f.write('\n')
f.write('------------------------- TRAINING -------------------------')
f.write('\n')
f.write(str(metrics.classification_report(y_train, y_train_pred, zero_division=0)))
f.write('------------------- Matrice di confusione ------------------')
f.write('\n')
f.write(str(metrics.confusion_matrix(y_train, y_train_pred)))
f.write('\n')

f.write('---------------------------- TEST ----------------------------')
f.write('\n')
f.write(str(metrics.classification_report(y_test, y_pred, zero_division=0)))
f.write('------------------- Matrice di confusione ------------------')
f.write('\n')
f.write(str(metrics.confusion_matrix(y_test, y_pred)))
f.write('\n')

In [ ]:
# Chiusura file
f.close()